In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

## Mock Draft Data

In [2]:
MOCK_DRAFT_LINK = 'https://fantasyfootballcalculator.com/mock-draft/results/format/ppr/teams/10?page='

In [3]:
def getDraftId(link):
    response = requests.get(link)
    html = response.text.encode()
    soup = BeautifulSoup(html,'html.parser')
    table = soup.find('table',{'class':'table mock-draft-results'})
    mockDrafts = [x['href'] for x in table.find_all('a',{'class':'btn btn-primary btn-sm'}) if x.text=='Mock Draft Results']
    return mockDrafts

def getMockDrafts(draftId):
    link = 'https://fantasyfootballcalculator.com'+draftId
    response=requests.get(link)
    html=response.text.encode()
    soup = BeautifulSoup(html,'html.parser')
    table = soup.find('tbody',{'id':'draftboard-body'})
    table = table.find_all('tr')
    table = [[player['title'] for player in row.find_all('a')] for row in table]
    draftOrder = [player for row in table for player in row]
    return draftOrder

def getDraftOrder(link):
    draftId=[]
    for i in range(1,41):
        draftId.extend(getDraftId(link+str(i)))
    allDrafts = [getMockDrafts(draft) for draft in draftId]
    return allDrafts

def getPlayerPickPositions(mockDrafts):
    playerDic = {player:[draft.index(player) for draft in mockDrafts if player in draft] for player in np.unique([player for draft in mockDrafts for player in draft])}
    return playerDic

def getSummaryStats(players):
    dic = {player: {'number_of_drafts':len(players[player]),
                    'earliest_pick':min(players[player]),
                    'latest_pick':max(players[player]),
                    '25th_percentile':np.percentile(players[player],25),
                    'med_pick':np.percentile(players[player],50),
                    '75th_percentile':np.percentile(players[player],75),
                    'avg_pick':np.mean(players[player]),
                    'standard_deviation':np.std(players[player],ddof=1),
                    'pearson_second_skewness_coef':(np.mean(players[player])-np.median(players[player]))/np.std(players[player],ddof=1)} for player in players}
    return pd.DataFrame.from_dict(dic,orient='index')

def createSummaryDataframe():
    mockDrafts = getDraftOrder(MOCK_DRAFT_LINK)
    playersPicked = getPlayerPickPositions(mockDrafts)
    df = getSummaryStats(playersPicked).reset_index()
    return df

In [4]:
data=createSummaryDataframe()

C:\Users\ibiyt\Anaconda3\lib\site-packages\numpy\core\_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
C:\Users\ibiyt\Anaconda3\lib\site-packages\numpy\core\_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\ibiyt\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in double_scalars


In [5]:
data=data.rename(columns={'index':'PLAYER'})

In [6]:
data.to_csv('playersDraftStatistics.csv',index=False)